### MATLAB mat files
The file `dmos.mat` has two arrays of length 982 each: dmos and orgs. `orgs(i)==0` for distorted images.
The arrays `dmos` and `orgs` are arranged by concatenating the dmos (and orgs) variables for each database as follows:

```dmos=[dmos_jpeg2000(1:227) dmos_jpeg(1:233) white_noise(1:174) gaussian_blur(1:174) fast_fading(1:174)]```
 
where `dmos_distortion(i)` is the dmos value for image `distortion/img_i.bmp` where distortion can be one of the five
described above. 

The values of dmos when corresponding `orgs==1` are zero (they are reference  images). Note that imperceptible
loss of quality does not necessarily mean a dmos value of zero due to the   nature of the score processing used.

The file refnames_all.mat contains a cell array refnames_all. Entry refnames_all{i} is the name of
the reference image for image i whose dmos value is given by dmos(i). If orgs(i)==0, then this is a valid

In [ ]:
import os
import re
from enum import unique, IntEnum

import numpy
import pandas
from scipy.io import loadmat


In [ ]:
paths = list(os.walk('/home/rocampo/data/live'))
files = {
    os.path.basename(path): [os.path.join(os.path.basename(path), file) for file in files if '.bmp' in file]
    for path, _, files in paths
    if len(list(filter(lambda x: '.bmp' in x, files))) > 0
}


In [ ]:
refnames_mat = loadmat('/home/rocampo/data/live/refnames_all.mat')
dmos_mat = loadmat('/home/rocampo/data/live/dmos.mat')
dmos_realigned_mat = loadmat('/home/rocampo/data/live/dmos_realigned.mat')



In [ ]:
refnames = numpy.hstack(refnames_mat['refnames_all'][0])
dmos = numpy.hstack(dmos_mat['dmos'])
dmos_realigned = numpy.hstack(dmos_realigned_mat['dmos_new'])
dmos_realigned_std = numpy.hstack(dmos_realigned_mat['dmos_std'])



In [ ]:
def extract_index(file_name):
    return int(re.findall(r'\d+', file_name)[-1])

def create_array(file_names):
    return {extract_index(file_name): file_name for file_name in file_names}

In [ ]:
@unique
class Distortion(IntEnum):
    jpeg_2000 = 1
    jpeg = 2
    white_noise = 3
    gaussian_blur = 4
    fast_fading = 5

arrays = {
    Distortion.jpeg_2000: create_array(files['jp2k']),
    Distortion.jpeg: create_array(files['jpeg']),
    Distortion.white_noise: create_array(files['wn']),
    Distortion.gaussian_blur:  create_array(files['gblur']),
    Distortion.fast_fading:  create_array(files['fastfading']),  
}

In [ ]:
dataframes = [
    pandas.DataFrame({
        'distortion': distortion, 
        'index': list(paths.keys()),
        'distorted_path': list(paths.values())
    })
    for distortion, paths in arrays.items() 
]

dataframe = pandas.concat(dataframes)

In [ ]:
dataframe = dataframe.sort_values(by=['distortion', 'index'])

In [ ]:
dataframe['reference_path'] = refnames
dataframe['dmos'] = dmos
dataframe['dmos_realigned'] = dmos_realigned
dataframe['dmos_realigned_std'] = dmos_realigned_std
dataframe.reference_path = 'refimgs/' + dataframe.reference_path
dataframe.distortion = dataframe.distortion.apply(lambda x: Distortion(x).name)

In [ ]:
dataframe.to_csv('/home/rocampo/data/live_metadata.csv', index=False)

